<a href="https://colab.research.google.com/github/Sogo95/Resume-de-texte/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objectif :
Mettre en pratique les concepts de modèle encoder-decoder en construisant un modèle capable de générer un résumé automatique d'un texte.


1.	Téléchargez et préparez un jeu de données : Utilisez un jeu de données contenant des textes longs ainsi que leurs résumés. Par exemple, vous pouvez utiliser le jeu de données CNN/Daily Mail disponible sur Hugging Face Datasets. Préparez les données en suivant ces étapes :
•	Charger les textes et les résumés associés.
•	Effectuer un prétraitement (nettoyage du texte, suppression des caractères spéciaux, etc.).
•	Diviser les données en ensembles d’entraînement, de validation et de test.


In [ ]:
pip install datasets

Charger les textes et les résumés associés.

In [ ]:
import os
from datasets import load_dataset

# Charger le jeu de données CNN/Daily Mail
ds = load_dataset("abisee/cnn_dailymail", '3.0.0')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
print(ds['train'][0]['article'])  # Texte de l'article
print(ds['train'][0]['highlights'])  # Résumé (highlights)

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [ ]:
# Afficher un exemple de données
print(ds['train'][0])  # Afficher le premier exemple

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
!pip install transformers

In [ ]:
!pip install nltk

In [ ]:
!pip install tensorflow


In [ ]:
import re
from nltk.stem import WordNetLemmatizer
import nltk
from datasets import load_dataset

# Télécharger les ressources pour la lemmatisation
nltk.download('wordnet')
nltk.download('punkt')

# Initialiser le lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction de nettoyage et lemmatisation
def netoy_lemmat(text):
    # Supprimer les balises HTML
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text)  # Supprimer les espaces multiples
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères spéciaux
    tokens = nltk.word_tokenize(text)  # Tokenisation
    lemmatized = [lemmatizer.lemmatize(token.lower()) for token in tokens]  # Lemmatisation
    return ' '.join(lemmatized)



# Appliquer le nettoyage et la lemmatisation
ds = ds.map(lambda x: {'article': netoy_lemmat(x['article']),
                        'highlights': netoy_lemmat(x['highlights'])})

# Diviser en ensembles d'entraînement, validation et test
train_data = ds['train']
val_data = ds['validation']
test_data = ds['test']

print("Traitement et division terminés.")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Traitement et division terminés.


    Construction du modèle Encoder-Decoder : Utilisez une architecture Transformer basée sur un modèle pré-entraîné tel que BART ou T5. Ces modèles sont particulièrement adaptés pour les tâches de résumé de texte.

    L'encoder reçoit le texte original et transforme les séquences d'entrée en une représentation cachée.
    Le decoder génère un résumé à partir de cette représentation cachée.

Charger le modèle pré-entraîné et le tokenizer

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer  # Pour BART
# from transformers import T5ForConditionalGeneration, T5Tokenizer  # Pour T5

import torch
from transformers import BartForConditionalGeneration, BartTokenizer

# Initialiser le tokenizer et le modèle BART
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Déplacer le modèle vers le GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

Préparation des données pour l'entrainement

In [ ]:
# Fonction de tokenisation
def tokenize_function(examples):
    inputs = tokenizer(examples['article'], truncation=True, padding='max_length', max_length=512)
    targets = tokenizer(examples['highlights'], truncation=True, padding='max_length', max_length=150)
    inputs['labels'] = targets['input_ids']
    return inputs

# Appliquer la tokenisation sur les ensembles d'entraînement et de validation
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Vérifier les données tokenisées
print(train_data[0])


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

{'article': 'london england reuters harry potter star daniel radcliffe gain access to a reported million million fortune a he turn on monday but he insists the money wont cast a spell on him daniel radcliffe a harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnist around the world the young actor say he ha no plan to fritter his cash away on fast car drink and celebrity party i dont plan to be one of those people who a soon a they turn suddenly buy themselves a massive sport car collection or something similar he told an australian interviewer earlier this month i dont think ill be particularly extravagant the thing i like buying are thing that cost about pound book and cd and dvd at radcliffe will be able to gamble in a casino buy a drink in a pub or see the horror film hostel part ii currently six place below his number one movie on the uk box office chart detail of how hell mark his landmark birthday are under wrap his agent and publicist

Entrainement du modèle

In [ ]:
from transformers import Trainer, TrainingArguments
# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',          # Évaluer après chaque époque
    learning_rate=3e-5,                   # Taux d'apprentissage
    per_device_train_batch_size=4,        # Taille du lot pour l'entraînement
    per_device_eval_batch_size=4,         # Taille du lot pour la validation
    num_train_epochs=3,                   # Nombre d'époques
    weight_decay=0.01,                    # Pénalité de poids
    save_total_limit=2,                   # Limiter le nombre de sauvegardes
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

# Entraîner le modèle
trainer.train()

print("Entraînement terminé.")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
import torch
torch.cuda.empty_cache()  # Libérer la mémoire GPU


In [ ]:
# Paramètres d'entraînement
batch_size = 2  # Réduire la taille du lot
num_epochs = 3
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Entraînement
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_data):
        optimizer.zero_grad()  # Réinitialiser les gradients


        with torch.cuda.amp.autocast():  # Activer la précision mixte
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels['input_ids'])
            loss = outputs.loss

        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids

        if i % 100 == 0:  # Afficher la perte tous les 100 lots
            print(f"Epoch {epoch}, Batch {i}, Loss: {loss.item()}")

print("Entraînement terminé.")

<ipython-input-39-bf7794369b1f>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Activer la précision mixte


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 6388 has 14.74 GiB memory in use. Of the allocated memory 14.56 GiB is allocated by PyTorch, and 43.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Envoyer le modèle au GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

from torch.utils.data import DataLoader
# Définir les hyperparamètres
batch_size = 4
num_epochs = 3
learning_rate = 5e-5

# Créer des DataLoader pour l'entraînement et la validation
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

# Optimiseur
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Entraînement du modèle
for epoch in range(num_epochs):
    model.train()  # Mettre le modèle en mode entraînement
    for batch in train_dataloader:
        optimizer.zero_grad()  # Réinitialiser les gradients

               # Calculer les pertes et faire la rétropropagation
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels['input_ids'])
        loss = outputs.loss
        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids

    print(f'Époque {epoch + 1}/{num_epochs} : Perte {loss.item()}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 6388 has 14.74 GiB memory in use. Of the allocated memory 14.56 GiB is allocated by PyTorch, and 43.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
print(torch.cuda.is_available())  # Devrait retourner True
print(torch.cuda.current_device())  # Affiche l'index du GPU utilisé


True
0


Évaluation : Une fois le modèle entraîné, évaluez ses performances en générant des résumés pour les textes du jeu de test. Utilisez des métriques comme ROUGE ou BLEU pour évaluer la qualité des résumés générés par rapport aux résumés de référence.

In [ ]:
model.eval()
with torch.no_grad():
    for batch in validation_loader:
        input_ids, label_ids = encode_data(batch['article'], batch['highlights'])
        outputs = model(input_ids=input_ids, labels=label_ids)
        val_loss = outputs.loss
        print(f"Perte de validation: {val_loss.item()}")
